In [1]:
## load packages
suppressPackageStartupMessages({
  library(DESeq2)
  library(emmeans)
  library(ggsignif)
  library(Hmisc)
  library(janitor)
  library(patchwork)
  library(phyloseq)
  library(scales)
  library(tidyverse)
  library(vegan)
  library(adespatial)
})

In [2]:
## paths to directories
repo <- file.path("/Users/abandla/Desktop/2_research/1_manuscripts/2_2020_brunei_peat_fire")
data <- file.path(repo, "1_data")
figures <- file.path(repo, "3_figures")

In [3]:
## set global theme options for plots
btp_theme <- theme(
  axis.text = element_text(size = 16, color = "black"),
  axis.text.y = element_text(margin = margin(0, 10, 0, 10)),
  axis.text.x = element_text(margin = margin(10, 0, 10, 0)),
  axis.title = element_text(size = 18),
  axis.ticks.length = unit(.25, "cm"),
  panel.border = element_rect(linewidth = 0.5, fill = NA),
  panel.background = element_rect(fill = NA),
  panel.grid = element_blank(),
  legend.text = element_text(size = 16),
  legend.title = element_text(size = 18),
  legend.key = element_rect(fill = NA),
  legend.background = element_rect(fill = NA)
)

In [4]:
## import phyloseq object
btp_fire_ps <- readRDS(file.path(data, "3_phyloseq", "2020_btp_fire_ps.rds"))
btp_fire_ps

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 3928 taxa and 24 samples ]
sample_data() Sample Data:       [ 24 samples by 6 sample variables ]
tax_table()   Taxonomy Table:    [ 3928 taxa by 7 taxonomic ranks ]

In [29]:
btp_fire_ps %>%
  subset_taxa(., Kingdom == "Archaea") %>%
  transform_sample_counts(., function(x) x / sum(x)) %>%
  psmelt %>%
  group_by(OTU, psf_type, depth, Kingdom, Phylum, Class) %>%
  reframe(Abundance = mean(Abundance)) %>%
  filter(OTU %in% c("ASV_4694"))

OTU,psf_type,depth,Kingdom,Phylum,Class,Abundance
<chr>,<fct>,<fct>,<chr>,<chr>,<chr>,<dbl>
ASV_4694,Burnt,0-5,Archaea,Halobacterota,Methanocellia,1.169682e-03
ASV_4694,Burnt,35-40,Archaea,Halobacterota,Methanocellia,2.826456e-04
ASV_4694,Burnt,95-100,Archaea,Halobacterota,Methanocellia,4.938272e-04
ASV_4694,Intact,0-5,Archaea,Halobacterota,Methanocellia,8.541168e-05
ASV_4694,Intact,35-40,Archaea,Halobacterota,Methanocellia,0.000000e+00
ASV_4694,Intact,95-100,Archaea,Halobacterota,Methanocellia,0.000000e+00


In [26]:
btp_fire_ps %>%
  subset_taxa(., Kingdom == "Archaea") %>%
  tax_table %>%
  data.frame %>%
  rownames_to_column("ASV") %>%
  filter(Phylum != "NA" & Phylum != "Nanoarchaeota") %>%
  select(ASV, Class) %>%
  write.csv("~/Desktop/arc_class.csv")

In [5]:
## hellinger-transformed archaeal counts
archaea_htf <- btp_fire_ps %>%
  subset_taxa(., Kingdom == "Archaea") %>%
  transform_sample_counts(., function(x) sqrt(x / sum(x))) %>%
  otu_table(.) %>%
  data.frame

In [6]:
## import environmental data
btp_env_depth <- read.csv(file.path(data, "1_metadata", "2020_btp_depth_env_data.csv"))

In [7]:
## subset and log-transform variable
btp_env_depth_std <- btp_env_depth %>%
  select(-one_of("psf_type", "depth", "TDS", "salinity")) %>%
  column_to_rownames("sample") %>%
  mutate_at(vars(-pH), log) %>%
  decostand(., method = "standardize")

In [8]:
## redundancy analysis
## which peat water variables explain compositional variability?
archaea_htf_rda <- rda(archaea_htf ~ ., btp_env_depth_std)
anova(archaea_htf_rda)
archaea_htf_rda_adjR2 <- RsquareAdj(archaea_htf_rda)$adj.r.squared

,Df,Variance,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>
Model,4,0.1489838,1.851254,0.023
Residual,19,0.3822669,NA,NA


In [9]:
## forward selection
## together, DO and temperature explain 13.3% variation
archaea_forward_sel <- forward.sel(Y = archaea_htf, X = btp_env_depth_std, nperm = 9999, verbose = FALSE)
archaea_forward_sel

Procedure stopped (alpha criteria): pvalue for variable 3 is 0.151300 (> 0.050000)


,variables,order,R2,R2Cum,AdjR2Cum,F,pvalue
,<I<chr>>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,DO,4,0.12031740,0.1203174,0.08033183,3.009020,0.0145
2,water_temperature,1,0.08841606,0.2087335,0.13337475,2.346538,0.0400


In [10]:
## hellinger-transformed bacterial counts
bacteria_htf <- btp_fire_ps %>%
  subset_taxa(., Kingdom == "Bacteria") %>%
  transform_sample_counts(., function(x) sqrt(x / sum(x))) %>%
  otu_table(.) %>%
  data.frame

In [11]:
## redundancy analysis
## which peat water variables explain compositional variability?
bacteria_htf_rda <- rda(bacteria_htf ~ ., btp_env_depth_std)
anova(bacteria_htf_rda)
bacteria_htf_rda_adjR2 <- RsquareAdj(bacteria_htf_rda)$adj.r.squared

,Df,Variance,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>
Model,4,0.1208650,1.465793,0.05
Residual,19,0.3916712,NA,NA


In [12]:
## forward selection
## together, DO and temperature explain 9.2% variation
## https://www.davidzeleny.net/anadat-r/doku.php/en:forward_sel_examples
## https://r.qcbs.ca/workshop10/book-en/exploration.html#exploration
bacteria_forward_sel <- forward.sel(Y = bacteria_htf, X = btp_env_depth_std, nperm = 9999, verbose = FALSE)
bacteria_forward_sel

Procedure stopped (alpha criteria): pvalue for variable 3 is 0.315500 (> 0.050000)


,variables,order,R2,R2Cum,AdjR2Cum,F,pvalue
,<I<chr>>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,water_temperature,1,0.09189847,0.09189847,0.05062113,2.226366,0.0232
2,DO,4,0.07946169,0.17136017,0.09244209,2.013777,0.0408
